# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from time import time

from data import FESDDataset
from data import Frame, AugmentationParams
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
sns.set()

import cv2

from model import FESD, train, val, test
import copy

import scipy

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

import datetime

from utils import AvgMeter, clip_gradient, get_scheduler
from utils.mode import Mode
from utils import err2gt, gt2err

from copy import deepcopy

from tqdm.notebook import tqdm

num_gpus = torch.cuda.device_count()
is_cuda = torch.cuda.is_available()
print(f"Num cuda GPUs: {num_gpus}")
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

Num cuda GPUs: 1


In [4]:
RECORDING_DIR = Path('H:/Recordings/')
CHECKPOINT_DIR = Path('checkpoints')

## Data Loading

Firstly we need to import all the recordings into the notebook.


In [5]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

In [6]:
batchsize = 10
im_size = 300

test_exercises = ['E-0.01', 'E-1.01', 'E-2.01', 'E-3.01']

dataset_train = FESDDataset(RECORDING_DIR, im_size, test_exercises, randomize_augmentation_params=True)
dataset_train.randomize_augmentation_params = True

dataset_test = FESDDataset(RECORDING_DIR, im_size, test_exercises, test=True)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batchsize)
test_loader = torch.utils.data.DataLoader(dataset_test)

Recordings Found: 17
Total Frames: 510
Recordings Found: 9
Total Frames: 270
51
270


In [7]:
joint_names_all = ["-", "Head", "Neck", "Torso", "Waist", "Left collar", "Left shoulder", "Left elbow", "Left wrist", "Left hand", "-", "Right collar", "Right shoulder", "Right elbow", "Right wrist", "Right hand", "-", "Left hip", "Left knee", "Left ankle", "-", "Right hip", "Right knee", "Right ankle", "-"]
joint_names = [i for i in joint_names_all if i != '-']

body_halves = np.array(["Upper Half", "Lower Half"])
limbs = np.array(["Head", "Torso", "Left arm", "Right arm", "Left leg", "Right leg"])

upper_body_i = [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
lower_body_i = [3, 14, 15, 16, 17, 18, 19]

torso_i     = [2, 3, 4, 9]
head_i      = [0, 1]
left_arm_i  = [5, 6, 7, 8]
right_arm_i = [10, 11, 12, 13]
left_leg_i  = [14, 15, 16]
right_leg_i = [17, 18, 19]

joint_errors = []
for je in joint_error_json:
  joint_errors.append(je["Name"])

In [8]:
i = 500
mode = Mode.FULL_BODY
dataset_train.mode = mode
rgb, depth, pose_2d, gt, session = dataset_train[i]
dataset_train.frame.show()

## Data Analysis

Now that the data is loaded we can analyse it

In [ ]:
cols = ['Type', 'Session', 'Difficulty', 'Exercise', 'Frame', 'Joint', 'Error', 'mode']
df_data = pd.DataFrame(columns=cols)

for i in tqdm(range(len(dataset_train))):
  for m in [Mode.FULL_BODY, Mode.HALF_BODY, Mode.LIMBS, Mode.JOINTS]:
    dataset_train.mode = m
    rgb, depth, pose_2d, gt, session = dataset_train[i]

    gt = gt2err(gt, m)[0]
    session, frame_i = dataset_train.get_index(i)
    frame = dataset_train.frame
    session_name = frame.session["Name"]
    exercise = frame.session['Session Parameters']['Exercise']
    difficulty = int(exercise[2])

    for j, err in enumerate(gt):
      row = ["Train", session_name, difficulty, exercise, frame_i, j, int(err), m]
      df_data.loc[len(df_data)] = row

for i in tqdm(range(len(dataset_test))):
  for m in [Mode.FULL_BODY, Mode.HALF_BODY, Mode.LIMBS, Mode.JOINTS]:
    dataset_test.mode = m
    _, _, _, gt, _ = dataset_test[i]
    gt = gt2err(gt, m)[0]
    session, frame_i = dataset_test.get_index(i)
    frame = dataset_test.frame
    session_name = frame.session["Name"]
    exercise = frame.session['Session Parameters']['Exercise']
    difficulty = int(exercise[2])

    for j, err in enumerate(gt):
      row = ["Test", session_name, difficulty, exercise, frame_i, j, int(err), m]
      df_data.loc[len(df_data)] = row

dataset_train.mode = mode
dataset_test.mode = mode
df_data

In [ ]:
df_data_joints    = df_data[df_data['mode'] == Mode.JOINTS]
df_data_limbs     = df_data[df_data['mode'] == Mode.LIMBS]
df_data_half_body = df_data[df_data['mode'] == Mode.HALF_BODY]
df_data_full_body = df_data[df_data['mode'] == Mode.FULL_BODY]

### Error Distribution Per Joint

Here we investigate the distribution of errors based on Body regions

In [ ]:
df_data_joints["pose_id"]      = df_data_joints["Session"] + "_" + df_data_joints["Frame"].astype(str)
df_data_joints["error_simple"] = df_data_joints["Error"].apply(lambda x: 1 if x > 0 else 0)
df_data_joints["joint_name"]   = df_data_joints["Joint"].apply(lambda x: joint_names[x])

In [ ]:
class_dict_halves = Mode.HALF_BODY.get_class_dict()
df_data_joints["Upper Body"] = df_data_joints["Joint"].apply(lambda x: x in class_dict_halves["Upper Body"])
df_data_joints["Lower Body"] = df_data_joints["Joint"].apply(lambda x: x in class_dict_halves["Lower Body"])

class_dict_limbs = Mode.LIMBS.get_class_dict()
df_data_joints["Torso"]     = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Torso"])
df_data_joints["Head"]      = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Head"])
df_data_joints["Left Arm"]  = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Left Arm"])
df_data_joints["Right Arm"] = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Right Arm"])
df_data_joints["Left Leg"]  = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Left Leg"])
df_data_joints["Right Leg"] = df_data_joints["Joint"].apply(lambda x: x in class_dict_limbs["Right Leg"])

In [ ]:
2340/780

In [ ]:
df_data_joints["body_part"].value_counts()/780

In [ ]:
df_data_joints["body_half"] = df_data_joints["Joint"].apply(lambda x: Mode.HALF_BODY.get_class(x))
df_data_joints["body_part"] = df_data_joints["Joint"].apply(lambda x: Mode.LIMBS.get_class(x))

In [ ]:
df_data_joints["body_part"].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
error_distr = df_data_joints.groupby("pose_id")["error_simple"].sum().reset_index()["error_simple"]
error_distr = error_distr[error_distr > 0]
error_distr.plot.hist(bins = range(1, 20), density=True)
error_distr.plot(kind = "kde", alpha = 0.7)
quant = error_distr.quantile(0.7)
ax.axvline(quant, color="red", alpha = .7, ymax = 1, linestyle = ":")
ax.set_title("Distribution of number of joints with error per pose")
ax.set_xlabel("Number of Joints with Error")
ax.set_ylabel("Number of poses")
ax.set_xlim(1, 20)
ax.set_xticks([1, 5, 10, 15, 20])
ax.set_yticks([])
plt.savefig("figures/distribution_of_joint_errors_per_pose.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize = (10,4), nrows=1, ncols=2)

err_dist_upper = df_data_joints[df_data_joints["body_half"] == "Upper Body"][df_data_joints["error_simple"] == 1].groupby("pose_id")["error_simple"].sum().reset_index()["error_simple"]
err_dist_lower = df_data_joints[df_data_joints["body_half"] == "Lower Body"].groupby("pose_id")["error_simple"].sum().reset_index()["error_simple"]

err_dist_upper = err_dist_upper[err_dist_upper > 0]
err_dist_lower = err_dist_lower[err_dist_lower > 0]

err_dist_upper.plot.hist(bins = range(1, 16), density=True, ax = axs[0])
err_dist_upper.plot(kind = "kde", ax = axs[0])

err_dist_lower.plot.hist(bins = range(1, 11), density=True, ax = axs[1])
err_dist_lower.plot(kind = "kde", ax = axs[1])

fig.suptitle("Distribution of number of joints with error per pose per body half")
axs[0].set_title("Upper Body")
axs[1].set_title("Lower Body")

axs[0].set_xlim(1, 15)
axs[0].set_xticks([1, 5, 10, 15])
axs[1].set_xlim(1, 10)
axs[1].set_xticks([1, 5, 10])

quant = err_dist_upper.quantile(0.7)
axs[0].axvline(quant, color="red", alpha = .7, ymax = 1, linestyle = ":")

quant = err_dist_lower.quantile(0.7)
axs[1].axvline(quant, color="red", alpha = .7, ymax = 1, linestyle = ":")


for ax in axs:
  ax.set_ylabel("Number of Errors")
  ax.set_xlabel("Number of Joints")
  ax.set_yticks([])

plt.savefig("figures/distribution_of_joint_errors_per_pose_per_body_half.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize = (6,8), nrows=3, ncols=2)

def plot_error_distribution(df, body_part, thresh_dict, ax):
  err_dist = df[df["body_part"] == body_part].groupby("pose_id")["error_simple"].sum().reset_index()["error_simple"]
  err_dist = err_dist[err_dist > 0]

  err_dist.plot.hist(bins = range(1, 11), density=True, ax = ax)
  err_dist.plot(kind = "kde", ax = ax)

  ax.set_title(body_part)

  quant = err_dist.quantile(0.7)
  thresh_dict[body_part] = quant
  ax.axvline(quant, color="red", alpha = .7, ymax = 1, linestyle = ":")

  ax.set_xlim(1, 5)
  ax.set_xticks([1, 2, 3, 4, 5])
  ax.set_yticks([])


thresh_dict = {}

plot_error_distribution(df_data_joints, "Torso",      thresh_dict, axs[0,0])
plot_error_distribution(df_data_joints, "Head",       thresh_dict, axs[0,1])
plot_error_distribution(df_data_joints, "Left Arm",   thresh_dict, axs[1,0])
plot_error_distribution(df_data_joints, "Right Arm",  thresh_dict, axs[1,1])
plot_error_distribution(df_data_joints, "Left Leg",   thresh_dict, axs[2,0])
plot_error_distribution(df_data_joints, "Right Leg",  thresh_dict, axs[2,1])

for ax in axs:
  ax[0].set_ylabel("Number of Errors")
  ax[1].set_ylabel("")
  
plt.tight_layout()
plt.savefig("figures/distribution_of_joint_errors_per_pose_per_body_part.png")
plt.show()

print(thresh_dict)

### Error Distribution

Here we investigate the distribution of errors over the dataset for different body regions

#### Joints

In [ ]:
df_data_joints["Joint Name"] = df_data_joints["Joint"].apply(lambda x: joint_names[x])
df_data_joints["Simple Error"] = df_data_joints["Error"] != 0
df_data_joints["Difficulty Name"] = df_data_joints["Difficulty"].apply(lambda x: "Trivial" if x == 0 else "Easy" if x == 1 else "Medium" if x == 2 else "Hard")
df_data_joints

In [ ]:
ax = df_data_joints[["Error", "Frame"]].groupby("Error").count().plot.pie(subplots=True, figsize=(4, 4), title="Error Distribution", labels=joint_errors, autopct='%1.1f%%', pctdistance=1.25, fontsize=12, explode=(.1,.1,.1,.1), labeldistance=None)

ax[0].set_ylabel("")
ax[0].legend(bbox_to_anchor=(1, 1.02), loc='center left')

plt.savefig("figures/dist_joints/Error_Distribution.png")
plt.plot()

In [ ]:
error_by_difficulty = df_data_joints[['Difficulty', "Difficulty Name", "Simple Error"]].groupby(["Difficulty Name"]).mean().sort_values(by="Difficulty")["Simple Error"]
ax = error_by_difficulty.plot.bar()
ax.set_ylabel("Error Rate")
ax.set_xlabel("Difficulty")
ax.set_title("Error Rate by Difficulty")
ax.set_ylim(0, 1)
labels = ['Trivial', 'Easy', 'Medium', 'Hard']
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(labels, rotation=0, ha='center')
plt.savefig("figures/dist_joints/Error_Rate_by_Difficulty.png")
ax

In [ ]:
err_group = df_data_joints[['Difficulty', "Difficulty Name", "Error"]].groupby(["Difficulty Name", "Error"]).count()["Difficulty"].unstack().T
error_distribution_by_difficulty = (err_group / err_group.sum()).T
error_distribution_by_difficulty['sort_key'] = pd.CategoricalIndex(error_distribution_by_difficulty, [3, 0, 2, 1])
error_distribution_by_difficulty.sort_values('sort_key', inplace=True)
error_distribution_by_difficulty.drop('sort_key', axis=1, inplace=True)

axs = error_distribution_by_difficulty.T.plot.pie(subplots=True, figsize=(10, 10), layout=(2, 2), legend=False, title="Error Distribution by Difficulty", labels=joint_errors, autopct='%1.1f%%', pctdistance=1.25, fontsize=12, explode=(.1,.1,.1,.1), sharex=False, sharey=False, labeldistance=None)

axs[0, 0].set_xlabel("Trivial")
axs[0, 0].set_ylabel("")
axs[0, 1].set_xlabel("Easy")
axs[0, 1].set_ylabel("")
axs[0, 1].legend(bbox_to_anchor=(1, 1.02), loc='center left')
axs[1, 0].set_xlabel("Medium")
axs[1, 0].set_ylabel("")
axs[1, 1].set_xlabel("Hard")
axs[1, 1].set_ylabel("")

plt.tight_layout()
plt.savefig("figures/dist_joints/Error_Distribution_by_Difficulty.png")
plt.plot()

In [ ]:
s = df_data_joints[["Joint", "Simple Error"]].groupby(["Joint"]).sum()
s["Sorted Names"] = joint_names
s = s.sort_values(by="Simple Error", ascending=False)

err_joints = df_data_joints[["Joint", "Joint Name", "Error"]]
error_distribution_by_joint = err_joints.groupby(["Joint", "Error"]).count().unstack().reindex(s.index).T

error_distribution_by_joint = (error_distribution_by_joint / error_distribution_by_joint.sum()).T.fillna(0)
error_distribution_by_joint.columns = error_distribution_by_joint.columns.droplevel()

error_distribution_by_joint.index = s["Sorted Names"].tolist()

edj = error_distribution_by_joint.T

fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(1, 1, 1)

axs = edj.plot.pie(ax=ax1, subplots=True, figsize=(10, 10), layout=(4, 5), legend=False, title="Error Distribution by Joint", fontsize=12, sharex=False, sharey=False, labeldistance=None, explode=(.05,.05,.05,.1))

for ax in axs.flatten():
  ax.set_xlabel(ax.get_ylabel())
  ax.set_ylabel("")

plt.legend(joint_errors, loc='upper left', bbox_to_anchor=(1, 1))

ax1.set_title("Error Distribution by Joint")
plt.savefig("figures/dist_joints/Error_Distribution_by_Joint.png")
plt.plot()

#### Full Body

In [ ]:
df_data_full_body["Joint Name"] = df_data_full_body["Joint"].apply(lambda x: joint_names[x])
df_data_full_body["Difficulty Name"] = df_data_full_body["Difficulty"].apply(lambda x: "Trivial" if x == 0 else "Easy" if x == 1 else "Medium" if x == 2 else "Hard")
df_data_full_body

In [ ]:
error_by_difficulty = df_data_full_body[['Difficulty', "Difficulty Name", "Error"]].groupby(["Difficulty Name"]).mean().sort_values(by="Difficulty")["Error"]
ax = error_by_difficulty.plot.bar()
ax.set_ylabel("Error Rate")
ax.set_xlabel("Difficulty")
ax.set_title("Error Rate by Difficulty")
ax.set_ylim(0, 1)
labels = ['Trivial', 'Easy', 'Medium', 'Hard']
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(labels, rotation=0, ha='center')
plt.savefig("figures/dist_full_body/Error_Rate_by_Difficulty.png")
ax

#### Body Halves

In [ ]:
df_data_half_body["Joint Name"] = df_data_half_body["Joint"].apply(lambda x: body_halves[x])
df_data_half_body["Difficulty Name"] = df_data_half_body["Difficulty"].apply(lambda x: "Trivial" if x == 0 else "Easy" if x == 1 else "Medium" if x == 2 else "Hard")
#df_data_half_body["Error"] = df_data_half_body[["Joint Name", "Error"]].apply(lambda x: x["Joint Name"] if x["Error"] == 1 else "No Error", axis=1)
df_data_half_body

In [ ]:
error_by_difficulty = df_data_half_body[['Difficulty', "Difficulty Name", "Error"]].groupby(["Difficulty Name"]).mean().sort_values(by="Difficulty")["Error"]
ax = error_by_difficulty.plot.bar()
ax.set_ylabel("Error Rate")
ax.set_xlabel("Difficulty")
ax.set_title("Error Rate by Difficulty")
ax.set_ylim(0, 1)
labels = ['Trivial', 'Easy', 'Medium', 'Hard']
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(labels, rotation=0, ha='center')
plt.savefig("figures/dist_half_body/Error_Rate_by_Difficulty.png")
ax

In [ ]:
s = df_data_half_body[["Joint", "Error"]].groupby(["Joint"]).sum()
s["Sorted Names"] = body_halves
s = s.sort_values(by="Error", ascending=False)

err_joints = df_data_half_body[["Joint", "Joint Name", "Error"]]
error_distribution_by_joint = err_joints.groupby(["Joint", "Error"]).count().unstack().reindex(s.index).T

error_distribution_by_joint = (error_distribution_by_joint / error_distribution_by_joint.sum()).T.fillna(0)
error_distribution_by_joint.columns = error_distribution_by_joint.columns.droplevel()

error_distribution_by_joint.index = s["Sorted Names"].tolist()

edj = error_distribution_by_joint.T

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(1, 1, 1)

axs = edj.plot.pie(ax=ax1, subplots=True, layout=(1, 2), legend=False, title="Error Distribution by Joint", fontsize=12, sharex=False, sharey=False, labeldistance=None, explode=(.05,.1), autopct='%1.1f%%', pctdistance=1.25)

for ax in axs.flatten():
  ax.set_xlabel(ax.get_ylabel())
  ax.set_ylabel("")

plt.legend(["No Error", "Error"], loc='upper left', bbox_to_anchor=(1, 1))

ax1.set_title("Error Distribution by Joint")
plt.savefig("figures/dist_half_body/Error_Distribution_by_Joint.png")
plt.plot()

#### Limbs

In [ ]:
df_data_limbs["Joint Name"] = df_data_limbs["Joint"].apply(lambda x: limbs[x])
df_data_limbs["Difficulty Name"] = df_data_limbs["Difficulty"].apply(lambda x: "Trivial" if x == 0 else "Easy" if x == 1 else "Medium" if x == 2 else "Hard")
df_data_limbs

In [ ]:
error_by_difficulty = df_data_limbs[['Difficulty', "Difficulty Name", "Error"]].groupby(["Difficulty Name"]).mean().sort_values(by="Difficulty")["Error"]
ax = error_by_difficulty.plot.bar()
ax.set_ylabel("Error Rate")
ax.set_xlabel("Difficulty")
ax.set_title("Error Rate by Difficulty")
ax.set_ylim(0, 1)
labels = ['Trivial', 'Easy', 'Medium', 'Hard']
ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(labels, rotation=0, ha='center')
plt.savefig("figures/dist_limbs/Error_Rate_by_Difficulty.png")
ax

In [ ]:
s = df_data_limbs[["Joint", "Error"]].groupby(["Joint"]).sum()
s["Sorted Names"] = limbs
s = s.sort_values(by="Error", ascending=False)

err_joints = df_data_limbs[["Joint", "Joint Name", "Error"]]
error_distribution_by_joint = err_joints.groupby(["Joint", "Error"]).count().unstack().reindex(s.index).T

error_distribution_by_joint = (error_distribution_by_joint / error_distribution_by_joint.sum()).T.fillna(0)
error_distribution_by_joint.columns = error_distribution_by_joint.columns.droplevel()

error_distribution_by_joint.index = s["Sorted Names"].tolist()

edj = error_distribution_by_joint.T

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(1, 1, 1)

axs = edj.plot.pie(ax=ax1, subplots=True, layout=(2, 3), legend=False, title="Error Distribution by Joint", fontsize=12, sharex=False, sharey=False, labeldistance=None, explode=(.05,.1), autopct='%1.1f%%', pctdistance=1.35)

for ax in axs.flatten():
  ax.set_xlabel(ax.get_ylabel())
  ax.set_ylabel("")

plt.legend(["No Error", "Error"], loc='upper left', bbox_to_anchor=(1, 1))

ax1.set_title("Error Distribution by Joint")
plt.savefig("figures/dist_limbs/Error_Distribution_by_Joint.png")
plt.plot()

## Model

Build the model according to the chosen mode

In [9]:
all_modes = True
mode = Mode.FULL_BODY

if all_modes:
  model_full_body = nn.DataParallel(FESD(Mode.FULL_BODY.get_num_layers()))
  model_half_body = nn.DataParallel(FESD(Mode.HALF_BODY.get_num_layers()))
  model_limbs     = nn.DataParallel(FESD(Mode.LIMBS.get_num_layers()))
  model_joints    = nn.DataParallel(FESD(Mode.JOINTS.get_num_layers()))
else:
  model = nn.DataParallel(FESD(mode.get_num_layers()))

if is_cuda:
  if all_modes:
    model_full_body = model_full_body.cuda()
    model_half_body = model_half_body.cuda()
    model_limbs     = model_limbs.cuda()
    model_joints    = model_joints.cuda()
  else:
    model = model.cuda()

### Train Model

In the following we define the training function and train a network on the training data.

In [10]:
if (all_modes):
  train_loader_full_body  = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.FULL_BODY, randomize_augmentation_params=True), batch_size=batchsize, shuffle=True)
  test_loader_full_body   = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.FULL_BODY, test=True))
  train_loader_half_body  = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.HALF_BODY, randomize_augmentation_params=True), batch_size=batchsize, shuffle=True)
  test_loader_half_body   = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.HALF_BODY, test=True))
  train_loader_limbs      = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.LIMBS, randomize_augmentation_params=True), batch_size=batchsize, shuffle=True)
  test_loader_limbs       = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.LIMBS, test=True))
  train_loader_joints     = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.JOINTS, randomize_augmentation_params=True), batch_size=batchsize, shuffle=True)
  test_loader_joints      = torch.utils.data.DataLoader(FESDDataset(RECORDING_DIR, im_size, test_exercises=test_exercises, mode=Mode.JOINTS, test=True))

else:
  train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batchsize)
  test_loader = torch.utils.data.DataLoader(dataset_test)

Recordings Found: 17
Total Frames: 510
Recordings Found: 9
Total Frames: 270
Recordings Found: 17
Total Frames: 510
Recordings Found: 9
Total Frames: 270
Recordings Found: 17
Total Frames: 510
Recordings Found: 9
Total Frames: 270
Recordings Found: 17
Total Frames: 510
Recordings Found: 9
Total Frames: 270


In [11]:
# epoch number
epochs = 100
# optimizer
optim = 'adam'
# learning rate
learning_rate = 0.000025
# learning rate scheduler. can be step, poly or cosine
lr_scheduler = 'cosine'
# warmup epoch
warmup_epoch = -1
# warmup multiplier
warmup_multiplier = 100
# for step scheduler. where to decay lr, can be a list
lr_decay_epochs = [120, 160, 200]
# for step scheduler. step size to decay lr
lr_decay_steps = 20 
# for step scheduler. decay rate for learning rate
lr_decay_rate = 0.01
# weight decay
weight_decay = 0.0001
# momentum for SGD
momentum = 0.9
# gradient clipping margin
clip = 0.5

In [12]:
n_data = len(train_loader.dataset)
CE = torch.nn.CrossEntropyLoss()

if mode == Mode.FULL_BODY:
    CE = torch.nn.CrossEntropyLoss()

if is_cuda:
    CE = CE.cuda()

if all_modes:
    if optim == 'adam':
        optimizer_full_body = torch.optim.Adam(model_full_body.parameters(),    learning_rate, weight_decay=weight_decay)
        optimizer_half_body = torch.optim.Adam(model_half_body.parameters(),    learning_rate, weight_decay=weight_decay)
        optimizer_limbs     = torch.optim.Adam(model_limbs.parameters(),        learning_rate, weight_decay=weight_decay)
        optimizer_joints    = torch.optim.Adam(model_joints.parameters(),       learning_rate, weight_decay=weight_decay)
    elif optim == 'adamW':
        optimizer_full_body = torch.optim.AdamW(model_full_body.parameters(),   learning_rate, weight_decay=weight_decay)
        optimizer_half_body = torch.optim.AdamW(model_half_body.parameters(),   learning_rate, weight_decay=weight_decay)
        optimizer_limbs     = torch.optim.AdamW(model_limbs.parameters(),       learning_rate, weight_decay=weight_decay)
        optimizer_joints    = torch.optim.AdamW(model_joints.parameters(),      learning_rate, weight_decay=weight_decay)
    elif optim == 'sdg':
        optimizer_full_body = torch.optim.SGD(model_full_body.parameters(),     learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)
        optimizer_half_body = torch.optim.SGD(model_half_body.parameters(),     learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)
        optimizer_limbs     = torch.optim.SGD(model_limbs.parameters(),         learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)
        optimizer_joints    = torch.optim.SGD(model_joints.parameters(),        learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)

    scheduler_full_body     = get_scheduler(optimizer_full_body, len(train_loader_full_body), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch,     warmup_multiplier)
    scheduler_half_body     = get_scheduler(optimizer_half_body, len(train_loader_half_body), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)
    scheduler_limbs         = get_scheduler(optimizer_limbs, len(train_loader_limbs), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)
    scheduler_joints        = get_scheduler(optimizer_joints, len(train_loader_joints), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)
else:
    if optim == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
    elif optim == 'adamW':
        optimizer = torch.optim.AdamW(model.parameters(), learning_rate, weight_decay=weight_decay)
    elif optim == 'sdg':
        optimizer = torch.optim.SGD(model.parameters(), learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)

    scheduler = get_scheduler(optimizer, len(train_loader), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)


In [13]:
if all_modes:
    model_iterator = list(zip([Mode.FULL_BODY,          Mode.HALF_BODY,         Mode.LIMBS,         Mode.JOINTS], 
                              [model_full_body,         model_half_body,        model_limbs,        model_joints], 
                              [optimizer_full_body,     optimizer_half_body,    optimizer_limbs,    optimizer_joints], 
                              [scheduler_full_body,     scheduler_half_body,    scheduler_limbs,    scheduler_joints],
                              [train_loader_full_body,  train_loader_half_body, train_loader_limbs, train_loader_joints], 
                              [test_loader_full_body,   test_loader_half_body,  test_loader_limbs,  test_loader_joints]))

In [23]:
model_columns = ["epoch", "iteration", "joint_id",
                  "gts", "preds", "confidences", 
                  "Avg loss", "loss", "accuracy", 
                  "tp", "tn", "fp", "fn", "precision", "recall", "f1", 
                  "cohens_kappa", "learning_rate",
                  "train_test", "exercise", "simplified", "mode"]
                  
df_model = pd.DataFrame(columns=model_columns)
pb = tqdm(range(1, epochs + 1), desc='Epoch')

for epoch in pb:
    if all_modes:    
        print(f"--- {epoch:3d} ---")
        for mode, model, optimizer, scheduler, train_loader, _ in model_iterator:
            tic = time()
            torch.cuda.empty_cache()
            
            loss = train(train_loader, model, optimizer, CE, scheduler, clip, epoch, epochs, is_cuda, mode, df_model)  
            pb.set_description(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.3e}, loss: {loss:.3e})')
        
            print(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.3e}, loss: {loss:.3e}, acc: {df_model.loc[len(df_model)-1]["accuracy"]:.3f}, f1: {df_model.loc[len(df_model)-1]["f1"]:.3f}, precision: {df_model.loc[len(df_model)-1]["precision"]:.3f}, recall: {df_model.loc[len(df_model)-1]["recall"]:.3f}, kappa: {df_model.loc[len(df_model)-1]["cohens_kappa"]:.3f}, time: {time() - tic:.2f}s)')      

            if (epoch) % 10 == 0:
                torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"{mode.name.lower()}_{epoch}_ckpt.pth")) 
    else:
        tic = time()
        torch.cuda.empty_cache()
        loss = train(train_loader, model, optimizer, CE, scheduler, clip, epoch, epochs, is_cuda, mode, df_model)

        pb.set_description(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.3e}, loss: {loss:.3e})')
        
        if (epoch) % 10 == 0:
            torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"{epoch}_ckpt.pth"))
    
if all_modes:
    for mode, model, _, _, _, _ in model_iterator:
        torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"{mode.name.lower()}_last_ckpt.pth")) 
        print(f"model saved {os.path.join(CHECKPOINT_DIR, f'last_ckpt.pth')}!")
else:
    torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"last_ckpt.pth"))
    print(f"model saved {os.path.join(CHECKPOINT_DIR, f'last_ckpt.pth')}!")
    checkpoint = os.path.join(CHECKPOINT_DIR, f"last_ckpt.pth")

df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

---   1 ---
Epoch (mode:  full body, lr: 2.491e-05, loss: -2.153e+04, acc: 0.500, f1: 0.667, precision: 0.500, recall: 1.000, kappa: 0.000, time: 46.19s)
Epoch (mode:  half body, lr: 2.491e-05, loss: -3.955e+04, acc: 0.800, f1: nan, precision: nan, recall: 0.000, kappa: 0.000, time: 45.21s)
Epoch (mode:      limbs, lr: 2.491e-05, loss: -6.938e+04, acc: 0.800, f1: 0.889, precision: 0.800, recall: 1.000, kappa: 0.000, time: 45.80s)
Epoch (mode:     joints, lr: 2.492e-05, loss: -3.837e+05, acc: 0.800, f1: nan, precision: nan, recall: 0.000, kappa: 0.000, time: 53.94s)
---   2 ---
Epoch (mode:  full body, lr: 2.485e-05, loss: -6.068e+04, acc: 0.900, f1: 0.947, precision: 0.900, recall: 1.000, kappa: 0.000, time: 48.78s)
Epoch (mode:  half body, lr: 2.486e-05, loss: -1.147e+05, acc: 0.400, f1: nan, precision: nan, recall: 0.000, kappa: 0.000, time: 49.22s)
Epoch (mode:      limbs, lr: 2.486e-05, loss: -2.037e+05, acc: 0.700, f1: 0.824, precision: 0.875, recall: 0.778, kappa: -0.154, time: 5

ArrowInvalid: ('Could not convert <Mode.FULL_BODY: 0> with type Mode: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column mode with type object')

### Test Model

In [24]:
if all_modes:
  for mode, model, _, _, _, test_loader in model_iterator:
    test(test_loader, model, CE, is_cuda, mode, df_model)
else: 
  test(test_loader, model, CE, is_cuda, mode, df_model)
  
df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 

ArrowInvalid: ('Could not convert <Mode.FULL_BODY: 0> with type Mode: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column mode with type object')

In [27]:
df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 

## Model Evaluation

Here we create all graphs and analysis for the training

In [ ]:
# Load the results of training and testing

df_model = pd.read_parquet('ModelAnalysis.parquet.gzip')
df_model["difficulty"] = df_model.apply(lambda x: int(x["exercise"][2]), axis=1)
print(df_model.columns)

In [ ]:
coi = ["difficulty", "accuracy", "tp", "tn", "fp", "fn", "precision", "recall", "f1", "cohens_kappa", "learning_rate"]

### Full Body Evaluation

The evaluation of the whole body as an error boolean.

In [ ]:
df_model_full_body = df_model[df_model["mode"] == "FULL_BODY"]

In [ ]:
df_model_half_body_test[df_model_half_body_test["simplified"] == False][coi].groupby(["difficulty"]).mean()

In [ ]:
df_model_half_body_test[df_model_half_body_test["simplified"] == False][coi].groupby(["difficulty"]).sum()

### Half Body Evaluation

The evaluation of the body split into two parts (upper and lower body), each as an error boolean.

In [ ]:
# Split the data

df_model_half_body = df_model[df_model["mode"] == "half_body"]
df_model_half_body["joint_names"] = df_model_half_body["joint_id"].apply(lambda x: body_halves[x])
df_model_half_body_test = df_model_half_body[df_model_half_body["train_test"] == "test"]

In [ ]:
df_model_half_body_test[df_model_half_body_test["simplified"] == False][coi + ["joint_names"]].groupby(["joint_names", "difficulty"]).mean()

### Limb Evaluation

The evaluation of the body split into limbs (Head and lower body), each as an error boolean.

In [ ]:
df_model["joint_name"] = df_model.apply(lambda x: joint_names[x["joint_id"]], axis=1)


In [ ]:
df_model.groupby(["epoch", "difficulty"]).mean()["accuracy"].plot()

In [ ]:
df_train = df_model[df_model['train_test'] == 'train']
df_test = df_model[df_model['train_test'] == 'test']
df_train.head()

In [ ]:
df_train[df_train["simplified"] == True].groupby("epoch").mean(numeric_only=True)["accuracy"].plot()

In [ ]:
df_test[df_test['simplified'] == True].groupby(["epoch", "joint_name"]).sum().sort_values("accuracy", ascending=False)

In [ ]:
loss = 0
for j in range(0, 20):
  print(gt, pred)
  loss += CE(gt[:,j], pred[:,j])
loss

In [ ]:
#np.array(gts[0])
gts[0]
gt = torch.tensor(np.array(json.loads(gts[5])))
pred = torch.tensor(np.array(json.loads(preds[5])))


In [ ]:
len(gts) * 20

In [ ]:
df_train_test_cp = df_train_test.copy()

In [ ]:
gts = np.array(df_train[df_train['simplified'] == False]["gts"].tolist())
preds = np.array(df_train[df_train['simplified'] == False]["preds"].tolist())
exercises = np.array(df_train[df_train['simplified'] == False]["exercise"].tolist())

model_columns_test = ["epoch", "iteration", "joint_id",
                  "gts", "preds", "confidences", 
                  "Avg loss", "loss", "accuracy", 
                  "tp", "tn", "fp", "fn", "precision", "recall", "f1", 
                  "cohens_kappa", "learning_rate",
                  "train_test", "exercise", "simplified"]

df_train_test = pd.DataFrame(columns=model_columns_test)

for it, (gt, pred, exercise) in tqdm(enumerate(zip(gts, preds, exercises)), total=len(gts)):
  i = it % len(train_loader)
  epoch = it // len(train_loader)

  gt = torch.tensor(np.array(json.loads(gt)))
  pred = torch.tensor(np.array(json.loads(pred)))
  
  loss = 0

  for j in range(0, 20):
    loss += CE(gt[:,j], pred[:,j])

  loss_record = AvgMeter()

  loss_record.update(loss.data, 1)

  val(pred, gt, loss_record, loss, np.NaN, epoch, epochs, i, len(train_loader), "train",  exercise, df_train_test)  

In [ ]:
gts = np.array(df_test[df_test['simplified'] == False]["gts"].tolist())
preds = np.array(df_test[df_test['simplified'] == False]["preds"].tolist())
exercises = np.array(df_test[df_test['simplified'] == False]["exercise"].tolist())

model_columns_test = ["epoch", "iteration", "joint_id",
                  "gts", "preds", "confidences", 
                  "Avg loss", "loss", "accuracy", 
                  "tp", "tn", "fp", "fn", "precision", "recall", "f1", 
                  "cohens_kappa", "learning_rate",
                  "train_test", "exercise", "simplified"]

df_test_test = pd.DataFrame(columns=model_columns_test)

for it, (gt, pred, exercise) in tqdm(enumerate(zip(gts, preds, exercises)), total=len(gts)):
  i = it % len(train_loader)
  epoch = it // len(train_loader)

  gt = torch.tensor(np.array(json.loads(gt)))
  pred = torch.tensor(np.array(json.loads(pred)))
  
  loss = 0

  for j in range(0, 20):
    loss += CE(gt[:,j], pred[:,j])

  loss_record = AvgMeter()

  loss_record.update(loss.data, 1)

  val(pred, gt, loss_record, loss, np.NaN, epoch, epochs, i, len(train_loader), "test",  exercise, df_test_test)  

In [ ]:
gt[:,0]

In [ ]:
df_test_test["joint_name"] = df_test_test.apply(lambda x: joint_names[x["joint_id"]], axis=1)

In [ ]:
df_test_test[df_test_test["exercise"] == "E-1.01"][df_test_test["simplified"] == True].reset_index()

In [ ]:
38*20

In [ ]:
tps = df_test_test[df_test_test["simplified"] == True].groupby(["exercise"]).sum(numeric_only=True)[["tp", "tn", "fp", "fn", "accuracy"]]
tps["positives"] = tps["tp"] + tps["fn"]
tps["negatives"] = tps["tn"] + tps["fp"]
tps["accuracy"] = tps["accuracy"] / (tps["positives"] + tps["negatives"])
tps.sort_values('accuracy', ascending= False).sort_index(level='exercise', sort_remaining=False)
pd.set_option('display.max_rows', None)
tps = tps.sort_values('accuracy', ascending= False).sort_index(level='exercise', sort_remaining=False)
tps

In [ ]:
df_train_test[df_train_test["simplified"] == True][df_train_test["train_test"] == 'train'].groupby(["joint_id"]).mean()